In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Importing Train and test data

In [2]:
train = pd.read_csv('train.csv')

In [3]:
test = pd.read_csv('test.csv')

In [4]:
train.shape


(1460, 81)

In [5]:
test.shape

# sample submission is for submitting predictions into competetion site

(1459, 80)

**With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.**

In [6]:
train.info

<bound method DataFrame.info of         Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0        1          60       RL         65.0     8450   Pave   NaN      Reg   
1        2          20       RL         80.0     9600   Pave   NaN      Reg   
2        3          60       RL         68.0    11250   Pave   NaN      IR1   
3        4          70       RL         60.0     9550   Pave   NaN      IR1   
4        5          60       RL         84.0    14260   Pave   NaN      IR1   
...    ...         ...      ...          ...      ...    ...   ...      ...   
1455  1456          60       RL         62.0     7917   Pave   NaN      Reg   
1456  1457          20       RL         85.0    13175   Pave   NaN      Reg   
1457  1458          70       RL         66.0     9042   Pave   NaN      Reg   
1458  1459          20       RL         68.0     9717   Pave   NaN      Reg   
1459  1460          20       RL         75.0     9937   Pave   NaN      Reg   

     LandContour Ut

# Pre-processing

In [7]:
# train and test data must have same preprocessing
# combine train adn test data into 1 dataframe 
# for combining datasets of dataframes the number of variables
# and the sequence of variables must be same


# add dependend variable temprorily to test data

In [8]:
test['SalePrice'] = 'test'

In [9]:
test.shape

(1459, 81)

In [10]:
# concatinating bothe dataframes
combineddf = pd.concat([train,test],axis=0)
#axis = 0 refers to rows

In [11]:
combineddf.info

<bound method DataFrame.info of         Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0        1          60       RL         65.0     8450   Pave   NaN      Reg   
1        2          20       RL         80.0     9600   Pave   NaN      Reg   
2        3          60       RL         68.0    11250   Pave   NaN      IR1   
3        4          70       RL         60.0     9550   Pave   NaN      IR1   
4        5          60       RL         84.0    14260   Pave   NaN      IR1   
...    ...         ...      ...          ...      ...    ...   ...      ...   
1454  2915         160       RM         21.0     1936   Pave   NaN      Reg   
1455  2916         160       RM         21.0     1894   Pave   NaN      Reg   
1456  2917          20       RL        160.0    20000   Pave   NaN      Reg   
1457  2918          85       RL         62.0    10441   Pave   NaN      Reg   
1458  2919          60       RL         74.0     9627   Pave   NaN      Reg   

     LandContour Ut

In [12]:
pd.set_option("display.max_rows",82)
combineddf.isnull().sum().sort_values(ascending = False)

PoolQC           2909
MiscFeature      2814
Alley            2721
Fence            2348
FireplaceQu      1420
LotFrontage       486
GarageFinish      159
GarageQual        159
GarageCond        159
GarageYrBlt       159
GarageType        157
BsmtExposure       82
BsmtCond           82
BsmtQual           81
BsmtFinType2       80
BsmtFinType1       79
MasVnrType         24
MasVnrArea         23
MSZoning            4
Functional          2
Utilities           2
BsmtHalfBath        2
BsmtFullBath        2
GarageArea          1
BsmtFinSF1          1
SaleType            1
GarageCars          1
BsmtUnfSF           1
Electrical          1
Exterior2nd         1
Exterior1st         1
KitchenQual         1
TotalBsmtSF         1
BsmtFinSF2          1
TotRmsAbvGrd        0
Fireplaces          0
Id                  0
BedroomAbvGr        0
PavedDrive          0
WoodDeckSF          0
OpenPorchSF         0
EnclosedPorch       0
3SsnPorch           0
ScreenPorch         0
PoolArea            0
MiscVal   

# Split data into object and numeric

In [13]:

objectcols = combineddf.select_dtypes(include=['object'])
numericcols = combineddf.select_dtypes(include=np.number)

#np.number selects both int and float

In [14]:
objectcols.shape

(2919, 44)

In [15]:
numericcols.shape

(2919, 37)

 missing values for numeric is mean and median and for objects mode and mode frequencies

In [16]:
objectcols.isnull().sum().sort_values(ascending = False)

PoolQC           2909
MiscFeature      2814
Alley            2721
Fence            2348
FireplaceQu      1420
GarageCond        159
GarageQual        159
GarageFinish      159
GarageType        157
BsmtCond           82
BsmtExposure       82
BsmtQual           81
BsmtFinType2       80
BsmtFinType1       79
MasVnrType         24
MSZoning            4
Utilities           2
Functional          2
Exterior2nd         1
Electrical          1
SaleType            1
Exterior1st         1
KitchenQual         1
RoofStyle           0
Neighborhood        0
SaleCondition       0
ExterQual           0
LotShape            0
LandContour         0
ExterCond           0
PavedDrive          0
LotConfig           0
LandSlope           0
Condition1          0
HouseStyle          0
Condition2          0
Foundation          0
RoofMatl            0
Street              0
CentralAir          0
HeatingQC           0
Heating             0
BldgType            0
SalePrice           0
dtype: int64

- if missing values are more then 70% of the column impute with the word missing or not available
- if missing values are less then 70% 
- numeric -- use mean and median 
- objects -- Use mode or most frequent


In [18]:
(objectcols.isnull().sum().sort_values(
    ascending = False)/objectcols.shape[0])*100

PoolQC           99.657417
MiscFeature      96.402878
Alley            93.216855
Fence            80.438506
FireplaceQu      48.646797
GarageCond        5.447071
GarageQual        5.447071
GarageFinish      5.447071
GarageType        5.378554
BsmtCond          2.809181
BsmtExposure      2.809181
BsmtQual          2.774923
BsmtFinType2      2.740665
BsmtFinType1      2.706406
MasVnrType        0.822199
MSZoning          0.137033
Utilities         0.068517
Functional        0.068517
Exterior2nd       0.034258
Electrical        0.034258
SaleType          0.034258
Exterior1st       0.034258
KitchenQual       0.034258
RoofStyle         0.000000
Neighborhood      0.000000
SaleCondition     0.000000
ExterQual         0.000000
LotShape          0.000000
LandContour       0.000000
ExterCond         0.000000
PavedDrive        0.000000
LotConfig         0.000000
LandSlope         0.000000
Condition1        0.000000
HouseStyle        0.000000
Condition2        0.000000
Foundation        0.000000
R

# Working With Object Columns

In [19]:
objectcols.columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition', 'SalePrice'],
      dtype='object')

In [20]:
not_available = ['Alley','FireplaceQu','MiscFeature','Fence','PoolQC']

- **Filling Missing Values as Not_available**

In [21]:
for col in not_available:
    objectcols[col] = objectcols[col].fillna("NotAvailable")
    
# in real time you have to do loops to fill missing values
    

In [22]:
garage_cols = [col for col in objectcols if col.startswith('Gar')]
garage_cols

['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']

In [23]:
for col in garage_cols:
    freq = objectcols[col].value_counts(dropna = False)
    print(freq)

Attchd     1723
Detchd      779
BuiltIn     186
NaN         157
Basment      36
2Types       23
CarPort      15
Name: GarageType, dtype: int64
Unf    1230
RFn     811
Fin     719
NaN     159
Name: GarageFinish, dtype: int64
TA     2604
NaN     159
Fa      124
Gd       24
Po        5
Ex        3
Name: GarageQual, dtype: int64
TA     2654
NaN     159
Fa       74
Gd       15
Po       14
Ex        3
Name: GarageCond, dtype: int64


In [24]:
Bsmt_cols = [col for col in objectcols if col.startswith('Bsmt')]
Bsmt_cols

['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']

In [25]:
for col in Bsmt_cols:
    freq = objectcols[col].value_counts(dropna = False)
    print(freq)

TA     1283
Gd     1209
Ex      258
Fa       88
NaN      81
Name: BsmtQual, dtype: int64
TA     2606
Gd      122
Fa      104
NaN      82
Po        5
Name: BsmtCond, dtype: int64
No     1904
Av      418
Gd      276
Mn      239
NaN      82
Name: BsmtExposure, dtype: int64
Unf    851
GLQ    849
ALQ    429
Rec    288
BLQ    269
LwQ    154
NaN     79
Name: BsmtFinType1, dtype: int64
Unf    2493
Rec     105
LwQ      87
NaN      80
BLQ      68
ALQ      52
GLQ      34
Name: BsmtFinType2, dtype: int64


frequency will decide that can we do mode imputation or not for missing values

Based on the freq analysis mode or most frequent imputation can be done for all missing values in object_cols

# Filling missing values with high frequency name using idxmax()

In [26]:
for col in objectcols.columns:
    objectcols[col] = objectcols[col].fillna(
    objectcols[col].value_counts().idxmax())
    
    # idxmax() - identifies the max frequency categoryname & uses it for imputation

# working with Numeric columns

In [27]:
numericcols.columns

#within numericcols their might be categorical cols like date 
#ratings , dummyencoded (0/1), etc
# we need to seperate categorical cols within the num cols into seperate dataframes


Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')

# Extracting categorical columns

In [28]:
category_cols = numericcols[['OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd','GarageYrBlt','MoSold', 'YrSold']]

In [29]:
numericcols = numericcols.drop(['OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd','GarageYrBlt','MoSold', 'YrSold'],axis=1)

In [30]:
print(numericcols.shape)
print(objectcols.shape)
print(category_cols.shape)

(2919, 30)
(2919, 44)
(2919, 7)


In [31]:
category_cols.isnull().sum().sort_values(ascending = False)

GarageYrBlt     159
OverallQual       0
OverallCond       0
YearBuilt         0
YearRemodAdd      0
MoSold            0
YrSold            0
dtype: int64

In [32]:
category_cols.GarageYrBlt=category_cols.GarageYrBlt.fillna(
category_cols.GarageYrBlt.value_counts().idxmax())

In [33]:
numericcols.isnull().sum().sort_values(ascending=False)

LotFrontage      486
MasVnrArea        23
BsmtFullBath       2
BsmtHalfBath       2
GarageCars         1
BsmtFinSF1         1
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
GarageArea         1
Id                 0
WoodDeckSF         0
ScreenPorch        0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
TotRmsAbvGrd       0
PoolArea           0
Fireplaces         0
FullBath           0
KitchenAbvGr       0
BedroomAbvGr       0
HalfBath           0
MSSubClass         0
GrLivArea          0
LowQualFinSF       0
2ndFlrSF           0
1stFlrSF           0
LotArea            0
MiscVal            0
dtype: int64

# Imputing missing values with median

In [34]:
# small number of missing values are there so we can impute with median

for col in numericcols.columns:
    numericcols[col] = numericcols[col].fillna(numericcols[col].median())

In [35]:
# Dummy variables encoding or label encoding of 
#object & category cols

objectcols.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition,SalePrice
0,RL,Pave,NotAvailable,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,RFn,TA,TA,Y,NotAvailable,NotAvailable,NotAvailable,WD,Normal,208500
1,RL,Pave,NotAvailable,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,RFn,TA,TA,Y,NotAvailable,NotAvailable,NotAvailable,WD,Normal,181500
2,RL,Pave,NotAvailable,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,RFn,TA,TA,Y,NotAvailable,NotAvailable,NotAvailable,WD,Normal,223500
3,RL,Pave,NotAvailable,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Unf,TA,TA,Y,NotAvailable,NotAvailable,NotAvailable,WD,Abnorml,140000
4,RL,Pave,NotAvailable,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,RFn,TA,TA,Y,NotAvailable,NotAvailable,NotAvailable,WD,Normal,250000


# Encoding the categorical Columns

In [36]:
from sklearn.preprocessing import LabelEncoder

In [37]:
lr = LabelEncoder()

In [38]:
numericcols['SalePrice']=objectcols.SalePrice

In [39]:
objectcols = objectcols.drop('SalePrice',axis=1)

In [40]:
objectcols_encoded = objectcols.apply(lr.fit_transform)

In [41]:
category_cols_encoded = category_cols.apply(lr.fit_transform)

In [42]:
numericcols.head()

# when numerical variables arre in different  mathematical units 
#use scaling techniques
#standard scalar or Z-Score -(x-mean)/stddev(both negativa and positive values)
# minmax scalar(positive range of 0 -- 1) and robust scalar(for outliers)

,Id,MSSubClass,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,SalePrice
0,1,60,65.0,8450,196.0,706.0,0.0,150.0,856.0,856,...,2.0,548.0,0,61,0,0,0,0,0,208500
1,2,20,80.0,9600,0.0,978.0,0.0,284.0,1262.0,1262,...,2.0,460.0,298,0,0,0,0,0,0,181500
2,3,60,68.0,11250,162.0,486.0,0.0,434.0,920.0,920,...,2.0,608.0,0,42,0,0,0,0,0,223500
3,4,70,60.0,9550,0.0,216.0,0.0,540.0,756.0,961,...,3.0,642.0,0,35,272,0,0,0,0,140000
4,5,60,84.0,14260,350.0,655.0,0.0,490.0,1145.0,1145,...,3.0,836.0,192,84,0,0,0,0,0,250000


# Calculating Z_score using standard Scalar

In [43]:
from sklearn.preprocessing import StandardScaler

In [44]:
scaler = StandardScaler()

In [45]:
numericcols_scaled = scaler.fit_transform(numericcols.drop(['Id','SalePrice'],axis=1))

In [46]:
numericcols_scaled = pd.DataFrame(numericcols_scaled,columns=numericcols.columns[1:30])

In [47]:
numericcols_scaled.head()

,MSSubClass,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,...,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal
0,0.067331,-0.191815,-0.217879,0.529034,0.580959,-0.293025,-0.934782,-0.444278,-0.773861,1.207379,...,-0.924311,0.306418,0.348888,-0.740760,0.200006,-0.359601,-0.103331,-0.285935,-0.06315,-0.089592
1,-0.873616,0.511940,-0.072044,-0.567016,1.178162,-0.293025,-0.629818,0.477158,0.261075,-0.785025,...,0.623632,0.306418,-0.059804,1.614879,-0.702843,-0.359601,-0.103331,-0.285935,-0.06315,-0.089592
2,0.067331,-0.051064,0.137197,0.338903,0.097928,-0.293025,-0.288441,-0.299027,-0.610718,1.235375,...,0.623632,0.306418,0.627542,-0.740760,-0.081209,-0.359601,-0.103331,-0.285935,-0.06315,-0.089592
3,0.302568,-0.426400,-0.078385,-0.567016,-0.494884,-0.293025,-0.047201,-0.671232,-0.506205,0.978742,...,0.623632,1.619830,0.785446,-0.740760,-0.184815,3.874967,-0.103331,-0.285935,-0.06315,-0.089592
4,0.067331,0.699608,0.518903,1.390216,0.468984,-0.293025,-0.160994,0.211621,-0.037170,1.671651,...,0.623632,1.619830,1.686426,0.776967,0.540424,-0.359601,-0.103331,-0.285935,-0.06315,-0.089592


In [48]:
objectcols_encoded = objectcols_encoded.reset_index(drop=True)

In [49]:
numericcols_scaled = numericcols_scaled.reset_index(drop=True)

In [50]:
category_cols_encoded = category_cols_encoded.reset_index(drop=True)

In [51]:
combineddf_clean = pd.concat([numericcols_scaled,objectcols_encoded,
                             category_cols_encoded],axis=1)

# Split data to train and test

In [52]:
numericcols = numericcols.reset_index(drop=True)

In [53]:
combineddf_clean['SalePrice'] = numericcols.SalePrice

In [54]:
combineddf_clean['SalePrice'] = numericcols.SalePrice

In [55]:
train_df = combineddf_clean[combineddf_clean.SalePrice!='test']

In [56]:
test_df = combineddf_clean[combineddf_clean.SalePrice=='test']

In [57]:
test_df = test_df.drop('SalePrice',axis=1)

In [58]:
y = train_df.SalePrice
x = train_df.drop('SalePrice',axis=1)

In [59]:
y = y.astype("int64")

# Building The Models

## Linear Regression

In [60]:
from sklearn.linear_model import LinearRegression

In [61]:
reg = LinearRegression()

In [62]:
regmodel = reg.fit(x,y)

In [63]:
regmodel.score(x,y)

0.8544531061204118

In [64]:
test_predict = regmodel.predict(test_df)

In [65]:
test_predict

array([106544.33985325, 161500.77319595, 166745.56202854, ...,
       142092.05182927, 113828.95028438, 242313.2592364 ])

In [66]:
pd.DataFrame(test_predict).to_csv("MLR.csv")

## Decision Tree Regressor

In [78]:
from sklearn.tree import DecisionTreeRegressor

In [79]:
tree = DecisionTreeRegressor()

In [80]:
treemodel = tree.fit(x,y)

In [81]:
treemodel.score(x,y)

1.0

In [82]:
treepredict = treemodel.predict(test_df)

In [84]:
treepredict

array([129000., 157900., 193000., ..., 142500., 108000., 219500.])

In [85]:
pd.DataFrame(treepredict).to_csv("DTR.csv")

 ## Random Forest Regressor

In [86]:
from sklearn.ensemble import RandomForestRegressor

In [87]:
rf = RandomForestRegressor(n_estimators=3000)

In [88]:
rfmodel = rf.fit(x,y)

In [89]:
rfmodel.score(x,y)

0.9823639182885214

In [90]:
rfpredict = rfmodel.predict(test_df)

In [92]:
rfpredict

array([126578.241     , 154505.90666667, 179745.988     , ...,
       152273.27733333, 114881.489     , 227743.097     ])

In [94]:
pd.DataFrame(rfpredict).to_csv("RF.csv")

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gbm = GradientBoostingRegressor(n_estimators=3000)

In [ ]:
gbmmodel = gbm.fit(x,y)

In [ ]:
gbmmodel.score(x,y)

# END